<a href="https://colab.research.google.com/github/QROST/AlpacaPlayground/blob/main/AlpacaTFSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: https://alpaca.markets/learn/tensorflow-market-forecasting/ 

### **Dataset Prep**

Install packages

In [1]:
!pip install alpaca-trade-api
!pip install --upgrade mplfinance
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Requirement already up-to-date: mplfinance in /usr/local/lib/python3.6/dist-packages (0.12.7a0)
(Reading database ... 144654 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import libraries

In [3]:
from datetime import datetime
import numpy as np
import talib
import alpaca_trade_api as tradeapi
import pandas

from time import sleep
from time import time
import datetime

import os
from os import listdir
from os.path import isfile, join

import argparse
import sys
import tempfile

import random

import tensorflow as tf
import tensorflow.compat.v1 as tfc
from sklearn import metrics

Initiate settings

In [4]:
# Creates dataset folders in directory script is run from
try:
    os.stat("./train")
    os.stat("./eval")
except BaseException:
    os.mkdir("./train")
    os.mkdir("./eval")

In [5]:
#paper trade account
api = tradeapi.REST(
    "PKW1DKS7S7VY873SRVCZ",
    "Xi6LgmUGtakBizsSVfb3o3os9Im4v7s0R8gOn4GO")

In [6]:
barTimeframe = "1D"  # 1Min, 5Min, 15Min, 1H, 1D

assetList = ['NVDA','MSFT','AMD','TRMB']

In [7]:
# ISO8601 date format
trainStartDate = "2000-01-01T00:00:00.000Z"
trainEndDate = "2017-06-01T00:00:00.000Z"
evalStartDate = "2017-06-01T00:00:00.000Z"
evalEndDate = "2020-10-01T00:00:00.000Z"

targetLookaheadPeriod = 1
startCutoffPeriod = 50  # Set to length of maximum period indicator

# Tracks position in list of symbols to download
iteratorPos = 0
assetListLen = len(assetList)

In [8]:
def CreateDataSet(iteratorPos):
    try:
        symbol = assetList[iteratorPos]
        # Returns market data as a pandas dataframe
        returned_data = api.get_barset(
            symbol,
            barTimeframe,
            start=trainStartDate,
            end=evalEndDate).df
        # Processes all data into numpy arrays for use by talib
        timeList = np.array(returned_data.index)
        openList = np.array(returned_data[symbol]['open'].tolist(), dtype=np.float64)
        print(openList)
        #print(returned_data)

        highList = np.array(returned_data[symbol]['high'].tolist(), dtype=np.float64)
        lowList = np.array(returned_data[symbol]['low'].tolist(), dtype=np.float64)
        closeList = np.array(returned_data[symbol]['close'].tolist(), dtype=np.float64)
        volumeList = np.array(returned_data[symbol]['volume'].tolist(), dtype=np.float64)
        # Adjusts data lists due to the reward function look ahead period
        shiftedTimeList = timeList[:-targetLookaheadPeriod]
        shiftedClose = closeList[targetLookaheadPeriod:]
        highList = highList[:-targetLookaheadPeriod]
        lowList = lowList[:-targetLookaheadPeriod]
        closeList = closeList[:-targetLookaheadPeriod]
        # Calculate trading indicators
        RSI14 = talib.RSI(closeList, 14)
        RSI50 = talib.RSI(closeList, 50)
        STOCH14K, STOCH14D = talib.STOCH(
            highList, lowList, closeList, fastk_period=14, slowk_period=3, slowd_period=3)
        # Calulate network target/ reward function for training
        closeDifference = shiftedClose - closeList
        closeDifferenceLen = len(closeDifference)
        # Creates a binary output if the market moves up or down, for use as
        # one-hot labels

        longOutput = np.zeros(closeDifferenceLen)
        longOutput[closeDifference >= 0] = 1
        shortOutput = np.zeros(closeDifferenceLen)
        shortOutput[closeDifference < 0] = 1
        # Constructs the dataframe and writes to CSV file
        outputDF = {
            "close": closeList,  # Not to be included in network training, only for later analysis
            "RSI14": RSI14,
            "RSI50": RSI50,
            "STOCH14K": STOCH14K,
            "STOCH14D": STOCH14D,
            "longOutput": longOutput,
            "shortOutput": shortOutput
        }
        # Makes sure the dataframe columns don't get mixed up
        columnOrder = ["close", "RSI14", "RSI50", "STOCH14K",
                        "STOCH14D", "longOutput", "shortOutput"]
        outputDF = pandas.DataFrame(
            data=outputDF,
            index=shiftedTimeList,
            columns=columnOrder)[
            startCutoffPeriod:]
        # Splits data into training and evaluation sets
        trainingDF = outputDF[outputDF.index < evalStartDate]
        evalDF = outputDF[outputDF.index >= evalStartDate]

        if (len(trainingDF) > 0 and len(evalDF) > 0):
            print("writing " + str(symbol) +
                  ", data len: " + str(len(closeList)))

            trainingDF.to_csv("./train/" + symbol + ".csv", index_label="date")
            evalDF.to_csv("./eval/" + symbol + ".csv", index_label="date")
    except BaseException:
        pass


In [10]:
#iterate through target stock symbols and create their data sets
while iteratorPos < assetListLen:   
    CreateDataSet(iteratorPos)   
    sleep(5)  # To avoid API rate limits
    iteratorPos += 1

[ 34.17  33.2   31.88 ... 521.61 517.5  526.3 ]
writing NVDA, data len: 3209
[ 35.79  35.24  35.22 ... 210.95 209.35 207.73]
writing MSFT, data len: 3209
[ 7.36    7.15    6.64   ... 79.1165 79.3    81.75  ]
writing AMD, data len: 3209
[15.265 14.765 14.625 ... 48.8   48.62  49.   ]
writing TRMB, data len: 3207


### **Training session**

In [11]:
# model settings
# Static seed to allow for reproducability between training runs
tf.random.set_seed(12345)
trainingCycles = 500000  # Number of training steps before ending
batchSize = 1000  # Number of examples per training batch
summarySteps = 1000  # Number of training steps between each summary
dropout = 0.5  # Node dropout for training
nodeLayout = [40, 30, 20, 10]  # Layout of nodes in each layer

In [12]:
mainDirectory = str("./model_1/")

trainFiles = [f for f in listdir("./train/") if isfile(join("./train/", f))]
evalFiles = [f for f in listdir("./eval/") if isfile(join("./eval/", f))]

In [13]:
# Initialises data arrays
trainDataX = np.empty([0, 4])
trainDataY = np.empty([0, 2])
evalDataX = np.empty([0, 4])
evalDataY = np.empty([0, 2])

In [14]:
# Reads training data into memory
readPos = 0
for fileName in trainFiles:
    importedData = pandas.read_csv("./train/" + fileName, sep=',')

    xValuesDF = importedData[["RSI14", "RSI50", "STOCH14K", "STOCH14D"]]
    yValuesDF = importedData[["longOutput", "shortOutput"]]

    xValues = np.array(xValuesDF.values.tolist())
    yValues = np.array(yValuesDF.values.tolist())

    trainDataX = np.concatenate([trainDataX, xValues], axis=0)
    trainDataY = np.concatenate([trainDataY, yValues], axis=0)

    #if readPos % 50 == 0 and readPos > 0: #why readPos % 50 == 0? need 50 or 100 stocks?
    if readPos > 0:
        print("Loaded " + str(readPos) + " training files")

    readPos += 1
print("\n\n")

Loaded 1 training files
Loaded 2 training files
Loaded 3 training files





In [15]:
# Reads evalutation data into memory
readPos = 0
for fileName in evalFiles:
    importedData = pandas.read_csv("./eval/" + fileName, sep=',')

    xValuesDF = importedData[["RSI14", "RSI50", "STOCH14K", "STOCH14D"]]
    yValuesDF = importedData[["longOutput", "shortOutput"]]

    xValues = np.array(xValuesDF.values.tolist())
    yValues = np.array(yValuesDF.values.tolist())

    evalDataX = np.concatenate([evalDataX, xValues], axis=0)
    evalDataY = np.concatenate([evalDataY, yValues], axis=0)

    #if readPos % 50 == 0 and readPos > 0:
    if readPos > 0:
        print("Loaded " + str(readPos) + " evaluating files")

    readPos += 1
print("\n\n")


Loaded 1 evaluating files
Loaded 2 evaluating files
Loaded 3 evaluating files





In [16]:
tfc.logging.set_verbosity(tfc.logging.INFO)

In [17]:
# used to sample batches from your data for training
def createTrainingBatch(amount):

    randomBatchPos = np.random.randint(0, trainDataX.shape[0], amount)

    xOut = trainDataX[randomBatchPos]
    yOut = trainDataY[randomBatchPos]

    return xOut, yOut

In [18]:
# ML training and evaluation functions
def train_tfc():
    globalStepTensor = tfc.Variable(0, trainable=False, name='global_step')

    sess = tfc.InteractiveSession()

    # placeholder for the input features
    x = tfc.placeholder(tfc.float32, [None, 4])
    # placeholder for the one-hot labels
    y = tfc.placeholder(tfc.float32, [None, 2])
    # placeholder for node dropout rate
    internalDropout = tfc.placeholder(tfc.float32, None)

    net = x  # input layer is the trading indicators

    # Creates the neural network model
    with tfc.name_scope('network'):
        # Initialises each layer in the network
        layerPos = 0
        for units in nodeLayout:
            net = tfc.layers.dense(
                net,
                units=units,
                activation=tfc.nn.tanh,
                name=str(
                    "dense" +
                    str(units) +
                    "_" +
                    str(layerPos)))  # adds each layer to the networm as specified by nodeLayout
            # dropout layer after each layer
            net = tfc.layers.dropout(net, rate=internalDropout)
            layerPos += 1

    logits = tfc.layers.dense(
        net, 2, activation=tfc.nn.softmax)  # network output

    with tfc.name_scope('lossFunction'):
        cross_entropy_loss = tfc.reduce_mean(
            tfc.nn.softmax_cross_entropy_with_logits_v2(
                labels=y,
                logits=logits))  # on NO account put this within a name scope - tensorboard shits itself

    with tfc.name_scope('trainingStep'):
        tfc.summary.scalar('crossEntropyLoss', cross_entropy_loss)
        trainStep = tfc.train.AdamOptimizer(0.0001).minimize(
            cross_entropy_loss, global_step=globalStepTensor)

    with tfc.name_scope('accuracy'):
        correctPrediction = tfc.equal(tfc.argmax(logits, 1), tfc.argmax(y, 1))
        accuracy = tfc.reduce_mean(tfc.cast(correctPrediction, tfc.float32))
        tfc.summary.scalar('accuracy', accuracy)

    merged = tfc.summary.merge_all()
    trainWriter = tfc.summary.FileWriter(
        mainDirectory + '/train', sess.graph, flush_secs=1, max_queue=2)
    evalWriter = tfc.summary.FileWriter(
        mainDirectory + '/eval', sess.graph, flush_secs=1, max_queue=2)
    tfc.global_variables_initializer().run()

    # Saves the model at defined checkpoints and loads any available model at
    # start-up
    saver = tfc.train.Saver(max_to_keep=2, name="checkpoint")
    path = tfc.train.get_checkpoint_state(mainDirectory)
    if path is not None:
        saver.restore(sess, tfc.train.latest_checkpoint(mainDirectory))

    lastTime = time()
    while tfc.train.global_step(sess, globalStepTensor) <= trainingCycles:
        globalStep = tfc.train.global_step(sess, globalStepTensor)

        # generates batch for each training cycle
        xFeed, yFeed = createTrainingBatch(batchSize)

        # Record summaries and accuracy on both train and eval data
        if globalStep % summarySteps == 0:
            currentTime = time()
            totalTime = (currentTime - lastTime)
            print(str(totalTime) + " seconds, " +
                  str(summarySteps / totalTime) + " steps/sec")
            lastTime = currentTime

            summary, accuracyOut, _ = sess.run([
                merged,
                accuracy,
                trainStep
            ],
                feed_dict={
                    x: xFeed,
                    y: yFeed,
                    internalDropout: dropout
                })
            trainWriter.add_summary(summary, globalStep)
            trainWriter.flush()
            print('Train accuracy at step %s: %s' % (globalStep, accuracyOut))

            summary, accuracyOut = sess.run([
                merged,
                accuracy,
            ],
                feed_dict={
                    x: evalDataX,
                    y: evalDataY,
                    internalDropout: 0
                })
            evalWriter.add_summary(summary, globalStep)
            evalWriter.flush()
            print('Eval accuracy at step %s: %s' % (globalStep, accuracyOut))

            print("\n\n")
            saver.save(sess, save_path=str(mainDirectory + "model"),
                       global_step=globalStep)  # saves a snapshot of the model

        else:  # Training cycle
            _ = sess.run(
                [trainStep], feed_dict={
                    x: xFeed,
                    y: yFeed,
                    internalDropout: dropout
                })

    trainWriter.close()
    evalWriter.close()

In [ ]:
%tensorboard --logdir logs

In [ ]:
tfc.disable_eager_execution()

#execute training //accuracy not ideal
train_tfc()